In [ ]:
!pip install flask-ngrok streamlit pyngrok scikit-learn --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.8 MB/s eta 0:00:00


In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
import joblib

X, y = load_iris(return_X_y=True)
model = LogisticRegression(max_iter=200)
model.fit(X, y)

joblib.dump(model, 'iris_model.pkl')


['iris_model.pkl']

In [ ]:
%%writefile backend.py
from flask import Flask, request, jsonify
import joblib
import numpy as np

app = Flask(__name__)
model = joblib.load("iris_model.pkl")

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    features = np.array(data['features']).reshape(1, -1)
    prediction = model.predict(features)[0]
    return jsonify({'prediction': int(prediction)})


Overwriting backend.py


In [ ]:
from pyngrok import ngrok

# Replace with your actual token
!ngrok config add-authtoken 2wXxfueSp56O25PLYNWXQVweS6G_4c16PAsbg7i4WY8zvzL61


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
import backend
import threading

# Start Flask server in thread
threading.Thread(target=lambda: backend.app.run(port=5000)).start()

# Open ngrok tunnel to port 5000 (Flask)
flask_url = ngrok.connect(5000)
print("Flask API URL:", flask_url)


 * Serving Flask app 'backend'
 * Debug mode: off
✅ Flask API URL: NgrokTunnel: "https://4d74-34-16-185-245.ngrok-free.app" -> "http://localhost:5000"


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [ ]:
%%writefile app.py
import streamlit as st
import requests

st.title("Iris Flower Classifier (via Flask API)")

sepal_length = st.slider("Sepal Length", 4.0, 8.0)
sepal_width = st.slider("Sepal Width", 2.0, 4.5)
petal_length = st.slider("Petal Length", 1.0, 7.0)
petal_width = st.slider("Petal Width", 0.1, 2.5)

if st.button("Predict"):
    features = [sepal_length, sepal_width, petal_length, petal_width]
    res = requests.post(
        "https://4d74-34-16-185-245.ngrok-free.app/predict",  # ← CHANGE this to your actual Flask ngrok URL
        json={"features": features}
    )
    if res.status_code == 200:
        prediction = res.json()["prediction"]
        st.success(f"Predicted class: {prediction}")
    else:
        st.error("Failed to get prediction from backend.")


Overwriting app.py


In [ ]:
# Start Streamlit and expose via ngrok
!nohup streamlit run app.py &>/dev/null &
public_url = ngrok.connect(8501)
print(" Streamlit app URL:", public_url)


Streamlit app URL: NgrokTunnel: "https://7cf1-34-16-185-245.ngrok-free.app" -> "http://localhost:8501"
